<a href="https://colab.research.google.com/github/ekrombouts/GenCareAI/blob/work_in_progress/scripts/work_in_progress/422_sampc_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

In [ ]:
# Load pre-trained fine-tuned model and tokenizer
model_finetuned = "ekrombouts/gcai_sampc_fietje"

model = AutoModelForCausalLM.from_pretrained(model_finetuned, torch_dtype=torch.bfloat16, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_finetuned)

# Set pad token
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# Load dataset
path_hf_sampc = "ekrombouts/Galaxy_SAMPC_long"
dataset = load_dataset(path_hf_sampc)
# We will be working with the validation dataset
val_dataset = dataset['validation']

In [ ]:
# Enable cache and set model to evaluation mode
model.config.use_cache = True
model.eval()


In [ ]:
def make_prompt(text, category=None):
    if category == "somatiek":
        prompt = f"""Lees de volgende rapportages en beschrijf de lichamelijke klachten van de cliënt.

Rapportages:
{text}

Geef de output als lijst van strings: ['foo', 'bar'].

Beschrijf de lichamelijke klachten:
"""
    elif category == "adl":
        prompt = f"""Lees de volgende rapportages en beschrijf welke hulp de cliënt nodig heeft bij wassen en kleden.

Rapportages:
{text}

Beschrijf de hulp bij wassen en kleden:
"""
    elif category == "continentie":
        prompt = f"""Lees de volgende rapportages en beschrijf de continentie van de cliënt.

Rapportages:
{text}

Beschrijf de continentie van de cliënt:
"""
    elif category == "mobiliteit":
        prompt = f"""Lees de volgende rapportages en beschrijf de mobiliteit van de cliënt.

Rapportages:
{text}

Beschrijf de mobiliteit van de cliënt:
"""
    elif category == "maatschappelijk":
        prompt = f"""Lees de volgende rapportages en beschrijf de bijzonderheden rondom familie en dagbesteding van de cliënt.

Rapportages:
{text}

Beschrijf de familie en dagbesteding:
"""
    elif category == "psychisch":
        prompt = f"""Lees de volgende rapportages en beschrijf de cognitie en gedragsproblemen van de cliënt.

Rapportages:
{text}

Geef de output als lijst van strings: ['foo', 'bar'].

Beschrijf cognitie en gedragsproblemen:
"""
    else:
        prompt = text
    return prompt

In [ ]:
def tokenize_prompt(prompt):
    # Tokenize and prepare input
    return tokenizer(prompt, return_tensors="pt").input_ids.to(model.device), \
           tokenizer(prompt, return_tensors="pt", padding=True).attention_mask.to(model.device)

def generate_output(input_ids, attention_mask):
    with torch.no_grad():
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=150,
            do_sample=True,
            top_p=0.95,
            top_k=50,
            temperature=0.8,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

def answer(notes, category=None):
    prompt = make_prompt(notes, category)
    input_ids, attention_mask = tokenize_prompt(prompt)
    generated_text = generate_output(input_ids, attention_mask)
    return generated_text[len(prompt):].strip()

In [ ]:
# Prepare the prompt with notes from sample
sample = val_dataset[13]
prompt = sample['prompt']
print(prompt)

# Display the generated response and actual response
ref_response = sample['reference']  # Reference response from dataset
print("GENERATED RESPONSE:")
print(answer(prompt))
print("\nREFERENCE RESPONSE:")
print(ref_response)

In [ ]:
rapportages = """U was inc van urine. U was niet vriendelijk tijdens het verschonen.
Mw was vanmorgen incontinent van dunne def, bed was ook nat. Mw is volledig verzorgd, bed is verschoond,
Mw. haar kledingkast is opgeruimd.
Mw. zei:"oooh kind, ik heb zo'n pijn. Mijn benen. Dat gaat nooit meer weg." Mw. zat in haar rolstoel en haar gezicht trok weg van de pijn en kreeg traanogen. Mw. werkte goed mee tijdens adl. en was vriendelijk aanwezig. Pijn. Mw. kreeg haar medicatie in de ochtend, waaronder pijnstillers. 1 uur later adl. gegeven.
Ik lig hier maar voor Piet Snot. Mw. was klaarwakker tijdens eerste controle. Ze wilde iets, maar wist niet wat. Mw. een slokje water gegeven en uitgelegd hoe ze kon bellen als ze iets wilde. Mw. pakte mijn hand en bedankte me.
Mevr. in de ochtend ondersteund met wassen en aankleden. Mevr was rustig aanwezig.
Mw is volledig geholpen met ochtendzorg, mw haar haren zijn gewassen. Mw haar nagels zijn kort geknipt.
Mevr heeft het ontbijt op bed genuttigd. Daarna mocht ik na de tweede poging Mevr ondersteunen met wassen en aankleden.
Vanmorgen met mw naar buiten geweest om een sigaret te roken. Mw was niet erg spraakzaam en mw kwam op mij over alsof ze geen behoefte had aan een gesprek. Mw kreeg het koud door de wind en wilde snel weer naar binnen.
"""

In [ ]:
answer(rapportages, "somatiek")

In [ ]:
answer(rapportages, "adl")

In [ ]:
answer(rapportages, "continentie")

In [ ]:
answer(rapportages, "mobiliteit")

In [ ]:
answer(rapportages, "maatschappelijk")

In [ ]:
answer(rapportages, "psychisch")

In [ ]:
prompt = f"""Lees de volgende rapportages.

Rapportages:
{rapportages}

Geef een korte samenvatting:
"""

In [ ]:
answer(prompt)

In [ ]:
# Prepare the prompt with truncated notes from sample
sample = val_dataset[13]
prompt = f"""Lees de volgende rapportages:

Rapportages:
{sample['notes']}

Beschrijf het eten en drinken:
"""
print(prompt)
print(100*"-")
answer(prompt)

In [ ]:
answer(sample['notes'], "somatiek")